In [4]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
env= gym.make('CartPole-v0')
# open Ai docs  

In [14]:
env 

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [15]:
epi =5
for epi in range(1,epi+1):
    state = env.reset()  # env.reset() :- reset the environment and obtain the observations
    done = False
    score = 0
    
    while not done:
        env.render() # render() :- visualize the environment
        action = env.action_space.sample() 
        n_state , reward , done , info = env.step(action) # apply action to the environment
        score+=reward
    print("Episodes {} , score {}".format(epi,score))
env.close() # close the render frame
    

Episodes 1 , score 18.0
Episodes 2 , score 9.0
Episodes 3 , score 29.0
Episodes 4 , score 33.0
Episodes 5 , score 18.0


In [16]:
env.step(1) # array is the state , reward = 0.0 , done state = true , info = {}

D:\Anaconda\lib\site-packages\gym\envs\classic_control\cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


(array([ 0.17515473,  1.4078608 , -0.2781185 , -2.4151757 ], dtype=float32),
 0.0,
 True,
 {})

In [17]:
env.action_space  # the action space

Discrete(2)

In [18]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [19]:
#training
logs_path = os.path.join('trainings','logs')

In [20]:
#recreating env
env= gym.make('CartPole-v0')
env = DummyVecEnv([lambda:env]) # wrapper for the env
model = PPO('MlpPolicy' , env , verbose=1 , tensorboard_log=logs_path) 
# PPO model with policy Mlp policy(multi layer perceptron)


Using cpu device


In [22]:
model.learn(total_timesteps=20000)


Logging to trainings\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1282 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 924         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006311087 |
|    clip_fraction        | 0.0205      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.584      |
|    explained_variance   | 0.699       |
|    learning_rate        | 0.0003      |
|    loss                 | 11.5        |
|    n_updates            | 60          |
|    policy_gradient_loss | -0.00414    |
|    value_loss           | 42          |
-----------------------------------------
--

In [21]:
ppo_save = os.path.join('trainings' , 'saved_model' , 'ppo_cartpole')

In [22]:
model.save(ppo_save)

In [23]:
del model

In [24]:
model = PPO.load(ppo_save , env=env)

In [27]:
#evaluate 

In [25]:
evaluate_policy(model , env , n_eval_episodes=10 , render=True)

D:\Anaconda\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(119.6, 15.49967741599805)

In [27]:
model.predict(obs) #model is predicting the action based on the obseervations

NameError: name 'obs' is not defined

In [28]:
action , _ = model.predict(obs)

NameError: name 'obs' is not defined

In [33]:
epi =5
for epi in range(1,epi+1):
    obs = env.reset()  # taking observations from the model 
    done = False
    score = 0
    while not done:
        env.render()
        action,_ = model.predict(obs) # using model 
        obs , reward , done , info = env.step(action) 
        score+=reward
    print("Episodes {} , score {}".format(epi,score))
env.close() 

Episodes 1 , score [200.]
Episodes 2 , score [200.]
Episodes 3 , score [200.]
Episodes 4 , score [200.]
Episodes 5 , score [200.]


In [35]:
#viewing logs 

In [34]:
trainings_logs = os.path.join(logs_path , 'PPO2')

In [ ]:
#!tensorboard --logdir={trainings_logs}

In [29]:
from stable_baselines3.common.callbacks import EvalCallback , StopTrainingOnRewardThreshold

In [30]:
save_path = os.path.join('trainings' , 'saved_model')

In [31]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200 , verbose=1)
eval_callback =EvalCallback(env , callback_on_new_best=stop_callback , eval_freq=10000 , best_model_save_path=save_path , verbose=1 )
# It will model will train utill it get the reward threshold , when it gets the model will be save in the given path.

In [33]:
model = PPO('MlpPolicy' , env , verbose=1 , tensorboard_log=logs_path)

Using cpu device


In [34]:
model.learn(total_timesteps=20000 , callback=eval_callback)

Logging to trainings\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 177  |
|    iterations      | 1    |
|    time_elapsed    | 11   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 272         |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008380398 |
|    clip_fraction        | 0.0898      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00374     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.67        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0144     |
|    value_loss           | 53.6        |
-----------------------------------------
--

D:\Anaconda\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0070801573 |
|    clip_fraction        | 0.0552       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.617       |
|    explained_variance   | 0.242        |
|    learning_rate        | 0.0003       |
|    loss                 | 22           |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0158      |
|    value_loss           | 67.4         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


In [ ]:
#using differant polices 

In [36]:
net_arch = [dict(pi=[128,128,128,128], # creating neural networks pi with four layers of unit 128,
                vf = [128,128,128,128])] # value function with four layer of unit 128 each )

In [41]:
model = PPO('MlpPolicy' , env , verbose=1 , tensorboard_log=logs_path , policy_kwargs={'net_arch' : net_arch}) # adding new policy


Using cpu device


In [42]:
model.learn(total_timesteps=20000 , callback=eval_callback)

Logging to trainings\logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 891  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 593         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014819006 |
|    clip_fraction        | 0.225       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00234    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.4         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.026      |
|    value_loss           | 19.4        |
-----------------------------------------
--

D:\Anaconda\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.012875166 |
|    clip_fraction        | 0.122       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.562      |
|    explained_variance   | 0.445       |
|    learning_rate        | 0.0003      |
|    loss                 | 13.1        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0138     |
|    value_loss           | 35.8        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 595   |
|    iterations      | 5     |
|    time_elapsed    | 17    |


In [43]:
from stable_baselines3 import DQN

In [44]:
model = DQN('MlpPolicy' , env , verbose=1 , tensorboard_log=logs_path)

Using cpu device


In [45]:
model.learn(total_timesteps=20000 , callback=eval_callback)

Logging to trainings\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.958    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1560     |
|    time_elapsed     | 0        |
|    total_timesteps  | 89       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.933    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2152     |
|    time_elapsed     | 0        |
|    total_timesteps  | 140      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.906    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 2442     |
|    time_elapsed     | 0        |
|    total_timesteps  | 198      |
----------------------------------
-----------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 5579     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2238     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 5653     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2313     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 5699     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2377     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 6192     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4614     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 6218     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4683     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 6255     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4811     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 6363     |
|    time_elapsed     | 1        |
|    total_timesteps  | 6982     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 6407     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7081     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 6415     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7142     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 6655     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9486     |
----------------------------------
Eval num_timesteps=9520, episode_reward=12.60 +/- 1.74
Episode length: 12.60 +/- 1.74
----------------------------------
| eval/               |          |
|    mean_ep_length   | 12.6     |
|    mean_reward      | 12.6     |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 9520     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 6500     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9576     |
----

...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 480      |
|    fps              | 6531     |
|    time_elapsed     | 1        |
|    total_timesteps  | 10616    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 484      |
|    fps              | 6542     |
|    time_elapsed     | 1        |
|    total_timesteps  | 10685    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 488      |
|    fps              | 6521     |
|    time_elapsed     | 1        |
|    total_timesteps  | 10756    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 588      |
|    fps              | 6566     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13089    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 592      |
|    fps              | 6568     |
|    time_elapsed     | 2        |
|    total_timesteps  | 13197    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 596      |
|    fps              | 6587     |
|    time_elapsed     | 2        |
|    total_timesteps  | 13288    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 696      |
|    fps              | 6563     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15341    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 700      |
|    fps              | 6552     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15420    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 704      |
|    fps              | 6570     |
|    time_elapsed     | 2        |
|    total_timesteps  | 15515    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 804      |
|    fps              | 6632     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17784    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 808      |
|    fps              | 6639     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17910    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 812      |
|    fps              | 6627     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17982    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 904      |
|    fps              | 6575     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19946    |
----------------------------------


In [ ]:
DQN.load()